In [1]:
df = spark.sql("SELECT * FROM bing_lake_db.tbl_latest_news")
display(df)

StatementMeta(, 594a92a5-7db1-49c6-b307-9f3e7403639b, 3, Finished, Available)

SynapseWidget(Synapse.DataFrame, 8890778d-041c-430f-9e54-57fb4e43b30b)

In [2]:
import synapse.ml.core
from synapse.ml.services import AnalyzeText

StatementMeta(, 594a92a5-7db1-49c6-b307-9f3e7403639b, 4, Finished, Available)

In [3]:
#Import the Model and Configure the input and output columns
model = (AnalyzeText()
        .setLocation("eastus")
        .setSubscriptionKey("7b6353b1d8fa44988e5fe4f85f9ae33c")
        .setTextCol("description")
        .setKind("SentimentAnalysis")
        .setOutputCol("response")
        .setErrorCol("error"))
        


StatementMeta(, 594a92a5-7db1-49c6-b307-9f3e7403639b, 5, Finished, Available)

In [4]:
#Apply the model to the df
result = model.transform(df)

StatementMeta(, 594a92a5-7db1-49c6-b307-9f3e7403639b, 6, Finished, Available)

In [5]:
display(result)

StatementMeta(, 594a92a5-7db1-49c6-b307-9f3e7403639b, 7, Finished, Available)

SynapseWidget(Synapse.DataFrame, 5916eabf-b2ea-4195-bf5a-8bc5b537a0ed)

In [6]:
#Create Sentiment Column
from pyspark.sql.functions import col

sentiment_df = result.withColumn('Sentiment', col('response.documents.sentiment'))

StatementMeta(, 594a92a5-7db1-49c6-b307-9f3e7403639b, 8, Finished, Available)

In [7]:
display(sentiment_df)

StatementMeta(, 594a92a5-7db1-49c6-b307-9f3e7403639b, 9, Finished, Available)

SynapseWidget(Synapse.DataFrame, 8d15f6f0-afb8-46e3-bb23-e244d627d9f2)

In [8]:
sentiment_df_final = sentiment_df.drop('error','response')

StatementMeta(, 594a92a5-7db1-49c6-b307-9f3e7403639b, 10, Finished, Available)

In [9]:
display(sentiment_df_final)

StatementMeta(, 594a92a5-7db1-49c6-b307-9f3e7403639b, 11, Finished, Available)

SynapseWidget(Synapse.DataFrame, 973b18ea-044d-4c24-bcce-5e89a5eb6cd1)

In [10]:
from pyspark.sql.utils import AnalysisException

try:

    table_name = 'bing_lake_db.tbl_sentiment_analysis'

    sentiment_df_final.write.format("delta").saveAsTable(table_name)

except AnalysisException:

    print("Table Already Existis")

    sentiment_df_final.createOrReplaceTempView("vw_sentiment_df_final")

    spark.sql(f""" Merge INTO {table_name} target_table
                   USING vw_sentiment_df_final source_view

                   ON source_view.url = target_table.url

                   WHEN MATCHED AND
                   source_view.title <> target_table.title OR
                   source_view.description <> target_table.description OR
                   source_view.category <> target_table.category OR
                   source_view.image <> target_table.image OR
                   source_view.provider <> target_table.provider OR
                   source_view.datePublished <> target_table.datePublished
                   
                   THEN UPDATE SET *

                   WHEN NOT MATCHED THEN INSERT * 

                   
                    """)


StatementMeta(, 594a92a5-7db1-49c6-b307-9f3e7403639b, 12, Finished, Available)

Table Already Existis
